In [1]:
# py-sc-fermi

import timeit

start = timeit.default_timer()

import numpy as np
from scipy.optimize import minimize_scalar
np.seterr(over='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
example_dir = '/Users/bjm42/source/sc-fermi/upload-sourceforge/example'

Open `unitcell.dat` to read in vectors and determine volume of pure cell

In [3]:
def read_unitcell_data(filename, verbose=True):
    with open(filename, 'r') as f:
        readin =  [ l for l in f.readlines() if l[0] != '#' ]
    factor = float(readin[0])
    lattvec = np.array([ [ float(s) for s in l.split() ] for l in readin[1:4] ], order='F')
    lattvec *= factor
    volume = ( lattvec[0,0] * (lattvec[1,1]*lattvec[2,2] - lattvec[1,2]*lattvec[2,1])
             + lattvec[0,1] * (lattvec[1,2]*lattvec[2,0] - lattvec[1,0]*lattvec[2,2])
             + lattvec[0,2] * (lattvec[1,0]*lattvec[2,1] - lattvec[1,1]*lattvec[2,0]) )
    if verbose:
        print(f"Volume of cell: {volume} A^3")
    return lattvec, volume


In [4]:
filename = f'{example_dir}/unitcell.dat'
lattvec, volume = read_unitcell_data(filename)

Volume of cell: 57.78888409009906 A^3


In [5]:
verbose = True
filename = f'{example_dir}/input-fermi.dat'
with open(filename, 'r') as f:
    readin = f.readlines()
    pure_readin = [ l for l in readin if l[0] != '#']
# print( ''.join(readin) )
nspinpol = int(pure_readin.pop(0))
if verbose:
    if nspinpol == 1:
        print( 'Found non-spin polarised system...' )
    elif nspinpol == 2:
        print( 'Found spin polarised system...' )
    else:
        raise ValueError('spin polarisation specification must be 1 or 2')
nelect = int(pure_readin.pop(0))
if verbose:
    print( f'Number of electrons in system: {nelect}')
egap = float(pure_readin.pop(0))
if verbose:
    print( f'Energy gap of system: {egap} eV')
    if(egap < 0.0):
        print("You have a negative gap - this is going to get weird...")
temperature = float(pure_readin.pop(0))
if verbose:
    print( f'Temperature: {temperature} K')
ndefects = int(pure_readin.pop(0))
if verbose:
    print( f'Number of defect species: {ndefects}')
    if ndefects < 0:
        print( '0 defects found...')
# For each species read in name, number of sites in the unit cell, 
# charge states, degeneracy and energies
name = []
ncharge = []
nsite = []
charge = []
energy = []
deg = []
for i in range(ndefects):
    l = pure_readin.pop(0).split()
    name.append( l[0])
    ncharge.append( int(l[1]) )
    nsite.append( int(l[2]) )
    if verbose:
        if ncharge[-1] <= 0:
            print(f"ERROR: defect {len[ndefects]+1} has idiotic number of charge states!!")
    charge.append([])
    energy.append([])
    deg.append([])
    for j in range(ncharge[-1]):
        l = pure_readin.pop(0).split()
        charge[-1].append(float(l[0]))
        energy[-1].append(float(l[1]))
        deg[-1].append(int(l[2]))
if verbose:
    print('Defects foung:')
    print('Name    # charge states    # sites in unit cell')
    for i in range(ndefects):
        print( name[i], ncharge[i], nsite[i])
        
# print(energy)



Found non-spin polarised system...
Number of electrons in system: 18
Energy gap of system: 0.8084 eV
Temperature: 100.0 K
Number of defect species: 2
Defects foung:
Name    # charge states    # sites in unit cell
V_Ga 4 1
Ga_Sb 3 1


In [6]:
# Read in density of states and renormalise. The energy scale should 
# be relative to the VBM (or Fermi level for a metal). Take into account
# spin polarisation specification


In [7]:
filename = f'{example_dir}/totdos.dat'
with open( filename, 'r') as f:
    readin = f.readlines()
    pure_readin = [ l.split() for l in readin if l[0] != '#']
# print( ''.join(readin) )
edos = []
dos = []
dosu = []
dosd = []
countdos = 0
if nspinpol == 1:
    for l in pure_readin:
        edos.append(float(l[0]))
        dos.append(float(l[1]))
        if dos[-1] < 0.0:
            countdos += 1
elif nspinpol == 2:
    for l in pure_readin:
        edos.append(float(l[0]))
        dosu.append(float(l[1]))
        dosd.append(float(l[2]))
        if (dosu < 0.0) or (dosd < 0.0):
            countdos += 1
        dos.append(dosu[-1] + dosd[-1])
if countdos > 0:
    print( "WARNING: Found negative value(s) of DOS!" )
    print("         Do you know what you are doing?")
    print("         These may cause serious problems...")
numdos = len(edos)
emin = edos[0]
emax = edos[-1]
if egap > emax:
    # BJM: Not sure why this check is here / implemented like this
    raise ValueError('ERROR: your conduction band is not present in energy range of DOS!!')

In [8]:
dos = np.array(dos)
edos = np.array(edos)

In [9]:
# For each defect calculate the transition levels and write them to 
# file for plotting. Determine them by finding the lowest energy defect
# at emin, then find earliest transition to a less positive charge state,
# and continue until all transitions are found

if ndefects > 0:
    with open('transition-levels.dat', 'w') as f:
        for i in range(ndefects):
            f.write(f'# {name[i]}\n')
            loweng = energy[i][0] + charge[i][0] * emin
            lownum = 0
            if ncharge[i] > 1:
                for j in range(ncharge[i]):
                    enertmp = energy[i][j] + charge[i][j] * emin
                    if enertmp - loweng < 0:
                        loweng = enertmp
                        lownum = j
            f.write( f'{emin} {loweng}\n' )
            count = 1
            while count > 0:
                count = 0
                transnum  = []
                fermitrans = []
                for j in range(ncharge[i]): # loop over all charge states of defect i
                    if j == lownum: 
                        continue
                    if (charge[i][j] < charge[i][lownum]):
                        count += 1
                        fermitrans.append( (energy[i][j]-energy[i][lownum]) / (charge[i][lownum]-charge[i][j]) )
                        transnum.append( j )
                if fermitrans:
                    loweng = fermitrans[0]
                    kcount = 0
                    for k in range(count):
                        if fermitrans[k] - loweng < 0:
                            loweng = fermitrans[k]
                            kcount = k
                    f.write( f'{fermitrans[kcount]} {energy[i][lownum] + charge[i][lownum] * fermitrans[kcount]}\n' )
                    lownum = transnum[kcount]
            f.write( f'{emax} {energy[i][lownum] + charge[i][lownum] * emax}\n')
            f.write('\n')

In [10]:
sum1 = 0.0
i = 0
while edos[i] <= 0.0:
    de = edos[i+1] - edos[i]
    sum1 += (dos[i] + dos[i+1]) * de / 2.0
    i += 1
#     if (i+1) > numdos:
#         raise ValueError('ERROR: Energy scale in DOS does not cross zero!!')
print(f'Integration of DOS up to Fermi level: {sum1}')

Integration of DOS up to Fermi level: 13.670273052867495


In [11]:
# vbm_index = np.where(edos <= 0)[0][-1]
# sum1 = np.trapz(dos[:vbm_index+2], edos[:vbm_index+2]) # BJM: possible off-by-one error?
# # np_edos[vbm_index+2] has a positive energy.

In [12]:
# normalise dos
dos = dos / sum1 * nelect

In [13]:
# sum1 = np.trapz(dos[:vbm_index+2], edos[:vbm_index+2])
# print(sum1)

In [14]:
# Now do algorithm to determine self-consistent Fermi level. Begin scanning
# at E_F = 0, go forwards by 1/10 of difference between E_F = 0 and E_F = 
# emax. Determine difference of (*) from zero, when sign of difference changes,
# reverse direction of scan reducing step size. Continue until step
# size is lower than a particular convergence criterion, or the difference
# is less than same. Take care for large exponentials (which return infinity),
# if they occur set the calculated value at a max. If this max is maintained
# between more than two steps, reverse the direction of the scan.
kboltz = 8.617343e-5
largefac = 700.0
vlarge = 1.014232054735e304
import math

ener0 = []
deg0 = []
if ndefects > 0:
    conc0 = []
    for i in range(ndefects):
        check0 = True
        for j in range(ncharge[i]):
            if charge[i][j] == 0:
                ener0.append( energy[i][j])
                deg0.append( deg[i][j] )
                check0 = False
        if check0:
            conc0.append(0.0)
        expfac = -ener0[-1] / (kboltz * temperature)
        if expfac > largefac:
            conc0.append(vlarge)
        else:
            conc0.append(nsite[i] * deg0[i] * math.exp(expfac))

In [15]:
conc0

[5.922742360775909e-124, 7.148137018479821e-115]

In [16]:
energy[0][1], expfac

(0.0265, -262.83043392841614)

In [17]:
kboltz

8.617343e-05

In [18]:
# set initial values for energy scan
estep = emax / 10.0
direction = 1.0
delphi = -estep

i=0
kmax = 0
kmin = 0
check_under = False
check_under2 = False
check_conv = False
finished = False

kT = kboltz * temperature

def p_func(delphi, dos, edos):
    return dos / (1.0 + np.exp((delphi - edos)/kT))

def n_func(delphi, dos, edos):
    return dos / (1.0 + np.exp((edos - delphi)/kT))

def carrier_concentrations(delphi, numdos, edos, dos):
    # get n0 and p0 using integrals (equations 28.9 in Ashcroft Mermin)
    p0_index = np.where(edos <= 0)[0][-1]
    n0_index = np.where(edos > egap)[0][0]
    p0 = np.trapz(p_func(delphi, dos[:p0_index+2], edos[:p0_index+2]), edos[:p0_index+2]) # BJM: possible off-by-one error?
    n0 = np.trapz(n_func(delphi, dos[n0_index:], edos[n0_index:]), edos[n0_index:])
    return p0, n0

def defect_charge_contributions(ndefects, ncharge, charge, energy, delphi):
    global concd
    lhs = 0.0
    rhs = 0.0
    concd = []
    # get defect concentrations at E_F
    if ndefects > 0:
        for j in range(ndefects):
            # Get concentrations at each charge
            concd.append([])
            for k in range(ncharge[j]):
                if charge[j][k] == 0:
                    concd[-1].append(0.0)
                else:
                    enerq = energy[j][k] + charge[j][k] * delphi
                    print(j,k,enerq,kT)
                    expfac = -enerq / kT
                    print(j,k,'expfac', expfac)
                    concd[j].append(nsite[j] * deg[j][k] * np.exp(expfac))
                    print('conc', j,  k, nsite[j] * deg[j][k] * np.exp(expfac))
                    if charge[j][k] < 0:
                        rhs += concd[j][k] * abs(charge[j][k])
                    else:
                        lhs += concd[j][k] * abs(charge[j][k])
                    print(lhs,rhs)
        print(lhs, rhs)
    return lhs, rhs
    
def q_tot(delphi, numdos, edos, dos, ndefects, ncharge, charge, energy):
    p0, n0 = carrier_concentrations(delphi, numdos, edos, dos)
    lhs_def, rhs_def = defect_charge_contributions(ndefects, ncharge, charge, energy, delphi)
    
    lhs = p0 + lhs_def
    rhs = n0 + rhs_def

    diff = rhs - lhs
    print(p0,n0)
    print(lhs,rhs)
    print(delphi)
    return diff

def abs_q_tot(delphi, numdos, edos, dos, ndefects, ncharge, charge, energy):
    return abs( q_tot(delphi, numdos, edos, dos, ndefects, ncharge, charge, energy))

In [19]:
# %%prun
conv = 1e-17

args = (numdos, edos, dos, ndefects, ncharge, charge, energy)
phi_min = minimize_scalar(abs_q_tot, method='bounded', bounds=(emin, emax), args=args, 
                tol=conv, options={'disp': False} )
print(phi_min)
delphi = phi_min.x

0 1 8.447110304088323 0.008617343000000001
0 1 expfac -980.2453382775087
conc 0 1 0.0
0.0 0.0
0 2 19.188120608176646 0.008617343000000001
0 2 expfac -2226.6864169357823
conc 0 2 0.0
0.0 0.0
0 3 27.97643091226497 0.008617343000000001
0 3 expfac -3246.526326300922
conc 0 3 0.0
0.0 0.0
1 1 10.514310304088323 0.008617343000000001
1 1 expfac -1220.1336658049147
conc 1 1 0.0
0.0 0.0
1 2 19.093920608176646 0.008617343000000001
1 2 expfac -2215.7549732181537
conc 1 2 0.0
0.0 0.0
0.0 0.0
6.310963993498592 0.0
6.310963993498592 0.0
-8.420610304088322
0 1 0.8829376570857792 0.008617343000000001
0 1 expfac -102.46054463490418
conc 0 1 3.176514574143959e-45
0.0 3.176514574143959e-45
0 2 4.059775314171558 0.008617343000000001
0 2 expfac -471.116829650573
conc 0 2 2.492071801239067e-205
0.0 3.176514574143959e-45
0 3 5.283912971257338 0.008617343000000001
0 3 expfac -613.1719453731082
conc 0 3 5.044378506878446e-267
0.0 3.176514574143959e-45
1 1 2.9501376570857794 0.008617343000000001
1 1 expfac -342.

conc 1 1 1.0505388628444778e-107
0.0 0.00159804513441543
1 2 2.310673698287492 0.008617343000000001
1 2 expfac -268.1422450385799
conc 1 2 3.5261648716730554e-117
0.0 0.00159804513441543
0.0 0.00159804513441543
0.001957173966270757 1.2574644599534008e-46
0.001957173966270757 0.00159804513441543
-0.028986849143745833
0 1 0.05163943975158093 0.008617343000000001
0 1 expfac -5.992501372126063
conc 0 1 0.002497409280748151
0.0 0.002497409280748151
0 2 2.397178879503162 0.008617343000000001
0 2 expfac -278.1807431250168
conc 0 2 1.5404169511232925e-121
0.0 0.002497409280748151
0 3 2.7900183192547425 0.008617343000000001
0 3 expfac -323.76781558477387
conc 0 3 2.4514569649753295e-141
0.0 0.002497409280748151
1 1 2.118839439751581 0.008617343000000001
1 1 expfac -245.88082889953208
conc 1 1 1.6417718432052684e-107
0.0 0.002497409280748151
1 2 2.302978879503162 0.008617343000000001
1 2 expfac -267.24929940738826
conc 1 2 8.611999082072497e-117
0.0 0.002497409280748151
0.0 0.002497409280748151


conc 1 1 1.2482255836406982e-107
0.0 0.0018987596662430153
1 2 2.30770209761379 0.008617343000000001
1 2 expfac -267.7974054895795
conc 1 2 4.9781104470083784e-117
0.0 0.0018987596662430153
0.0 0.0018987596662430153
0.0018987556811683765 1.4940897142851868e-46
0.0018987556811683765 0.0018987596662430153
-0.02750104880689501
0 1 0.05400102400840097 0.008617343000000001
0 1 expfac -6.266551535479203
conc 0 1 0.001898765130392233
0.0 0.001898765130392233
0 2 2.4019020480168023 0.008617343000000001
0 2 expfac -278.7288434517231
conc 0 2 8.904331943543026e-122
0.0 0.001898765130392233
0 3 2.7971030720252026 0.008617343000000001
0 3 expfac -324.5899660748333
conc 0 3 1.0773798593403803e-141
0.0 0.001898765130392233
1 1 2.121201024008401 0.008617343000000001
1 1 expfac -246.15487906288527
conc 1 1 1.248229175717611e-107
0.0 0.001898765130392233
1 2 2.307702048016802 0.008617343000000001
1 2 expfac -267.79739973409454
conc 1 2 4.978139098530583e-117
0.0 0.001898765130392233
0.0 0.0018987651303

/Users/bjm42/.pyenv/versions/3.7.0/lib/python3.7/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)


In [20]:
carrier_concentrations(-3.791978135534054, numdos, edos, dos), sum(edos), sum(dos)

((3.7171809174213264, 2.8398542928893414e-236),
 -6034.719698743704,
 965.8277187851861)

In [21]:
kT

0.008617343000000001

In [22]:
defect_charge_contributions(ndefects, ncharge, charge, energy, delphi)
print('Results:')
print('--------')
print(f'SC Fermi level :      {delphi}  (eV)')
p0, n0 = carrier_concentrations(delphi, numdos, edos, dos)
print( 'Concentrations:')
print( f'n (electrons)  : {n0* 1e24 / volume} cm^-3')
print( f'p (holes)      : {p0 * 1e24 / volume} cm^-3')
concall = []
if ndefects > 0:
    for i in range(ndefects):
        concall.append( conc0[i] )
        for j in range(ncharge[i]):
            concall[-1] += concd[i][j]
        print( f'{name[i]}      : {concall[-1]*1e24 / volume} cm^-3')
print()
print('Breakdown of concentrations for each defect charge state:')
for i in range(ndefects):
    print('---------------------------------------------------------')
    if concall[i] == 0:
        print( f'{name[i]}      : Zero total - cannot give breakdown')
        continue
    print(f'{name[i]}      : Charge Concentration(cm^-3) Total')
    for j in range(ncharge[i]):
        if charge[i][j] == 0:
            print(f'           : {charge[i][j]} {conc0[i]*1e24 / volume} {conc0[i]*100 / concall[i]}')
        else:
            print(f'           : {charge[i][j]} {concd[i][j]*1e24 / volume} {concd[i][j]*100 / concall[i]}')

0 1 0.05400106425324955 0.008617343000000001
0 1 expfac -6.2665562056946715
conc 0 1 0.0018987562627706567
0.0 0.0018987562627706567
0 2 2.4019021285064994 0.008617343000000001
0 2 expfac -278.72885279215404
conc 0 2 8.904248773633777e-122
0.0 0.0018987562627706567
0 3 2.7971031927597485 0.008617343000000001
0 3 expfac -324.58998008547974
conc 0 3 1.0773647646578192e-141
0.0 0.0018987562627706567
1 1 2.1212010642532495 0.008617343000000001
1 1 expfac -246.1548837331007
conc 1 1 1.2482233462320819e-107
0.0 0.0018987562627706567
1 2 2.307702128506499 0.008617343000000001
1 2 expfac -267.7974090745255
conc 1 2 4.97809260078323e-117
0.0 0.0018987562627706567
0.0 0.0018987562627706567
Results:
--------
SC Fermi level :      -0.02750106425324955  (eV)
Concentrations:
n (electrons)  : 2.5854228883234905e-24 cm^-3
p (holes)      : 3.2856773709964743e+19 cm^-3
V_Ga      : 3.2856773282043175e+19 cm^-3
Ga_Sb      : 2.159971485632336e-85 cm^-3

Breakdown of concentrations for each defect charge st

In [23]:
concd[0][1]

0.0018987562627706567

In [24]:
energy[0][1]-delphi

0.05400106425324955

In [25]:
expfac = -(energy[0][1]-delphi)/kT
expfac

-6.2665562056946715

In [26]:
np.exp(expfac)

0.0018987562627706567

In [27]:
nsite[0] * deg[0][1] * np.exp(expfac)

0.0018987562627706567

In [28]:
energy[0][1]-delphi

0.05400106425324955